# Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Multilabel

In [ ]:
!pip install scikit-multilearn

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

from numpy import array
from numpy import asarray
from numpy import zeros

from tensorflow.keras.preprocessing.text import Tokenizer


import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn import metrics
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,StratifiedKFold

from tensorflow.keras import initializers
from tensorflow.keras.constraints import max_norm

from keras.optimizer_v2 import adam
from skmultilearn.model_selection import iterative_train_test_split

seed = 7
np.random.seed(seed)

In [ ]:
#Read the data
#train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep learning/Trabajo final/processed_data.csv')
#train_df = pd.read_csv('processed_data.csv')
train_df = pd.read_csv('/kaggle/input/processed-data/processed_data.csv')

train_df = train_df.dropna()
X,y = pd.DataFrame(train_df.comment_text), train_df[['obscene','insult','toxic','severe_toxic','identity_hate','threat']]

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)
X_train = X_train.astype(str).values.reshape(-1).tolist()
X_test = X_test.astype(str).values.reshape(-1).tolist()
y_train = y_train.values
y_test = y_test.values

In [ ]:
vocab_size = 20000

#Tokenizer 
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

#glove_file = open('/content/drive/MyDrive/Colab Notebooks/Deep learning/Trabajo final/glove.6B.100d.txt', encoding="utf8")
glove_file = open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.100d.txt', encoding="utf8")
#glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    if index> vocab_size -1: #bounding by max feat
        break
    else:
        embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.1,
                              patience=8, min_lr=1e-10,verbose = 1)

custom_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', verbose=1, patience=20
)

num_folds = 6

# Define the K-fold Cross Validator
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)
kfold.get_n_splits(X_train, y_train)
# K-fold Cross Validation model evaluation
fold_no = 1
acc_per_fold = []
loss_per_fold = []

METRICS = [ 
      tf.keras.metrics.AUC(name='auc',multi_label = True),
]

In [ ]:
class_weight = {0:0.5278628291488059,1: 9.472527472527473,2:10.165418326693228,3:5.2366800755274605,4:51.56669361358124,5:56.49955713020372,6:172.4}

In [ ]:
# parameter settings
filters = 250
kernel_size = 3
hidden_dims = 250
batch_size = 128
n_epochs = 100
actfn = 'relu'
optimizer_ = tf.optimizers.Adam
drop_rate = 0.4
learningrate = 0.02
embedding_dim = 100
max_features = 20000

In [ ]:
embedding_matrix.shape

In [ ]:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
# function to build the model 
# function to build the model 
def model_cnn():
    
    model = Sequential()
    
    model.add(Embedding(max_features, embedding_dim, trainable = False
                       ,embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix)))    

    model.add(Dropout(0.4))
    
    model.add(Conv1D(filters, kernel_size, padding = 'valid')) #no padding
    
    model.add(MaxPooling1D())
    
    model.add(Conv1D(filters, 5, padding = 'valid', activation = actfn))
    
    model.add(GlobalMaxPooling1D())
    
    model.add(Dropout(0.4))
    
    model.add(Dense(250,activation = actfn))
    
    model.add(Dropout(0.4))
    
    model.add(Dense(6,activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy',    
                 optimizer = optimizer_(learning_rate = 0.001),
                 metrics = METRICS) 

    return model

In [ ]:
from tensorflow.keras.utils import plot_model
model = model_cnn()
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [ ]:
from skmultilearn.model_selection import IterativeStratification
fold_no = 1
k_fold = IterativeStratification(n_splits=5, order=6)
for train, test in k_fold.split(X_train, y_train):
    # Compile the model
    
    # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train[train], y_train[train], class_weight=class_weight, validation_data=(X_train[test], y_train[test]),batch_size=128, epochs=150,callbacks=[custom_early_stopping, reduce_lr])

    scores = model.evaluate(X_train[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import roc_auc_score

y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
clases = ['No toxic', 'Toxic']
#y_test = y_test.values
vis_arr = metrics.multilabel_confusion_matrix(y_train, y_pred)
labels = ["".join("c" + str(i)) for i in range(0, 6)]
print('ROC AUC: %.3f' % roc_auc_score(y_train, y_pred, average='weighted'))

In [ ]:
def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )

    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt=".3F", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for the class - " + class_label)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 7))
    
for axes, cfs_matrix, label in zip(ax.flatten(), vis_arr, labels):
    print_confusion_matrix(np.around(cfs_matrix / cfs_matrix.sum(axis=1)[:, np.newaxis],3), axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()

In [ ]:

def model_cnn():
    
    model = Sequential()
    
    model.add(Embedding(max_features, embedding_dim, trainable = False
                       ,embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix)))    
    model.add(Dropout(0.4))
    
    model.add(Conv1D(filters, kernel_size, padding = 'valid')) #no padding
    
    model.add(MaxPooling1D())
    
    model.add(Conv1D(filters, 5, padding = 'valid', activation = actfn))
    
    model.add(GlobalMaxPooling1D())
    
    model.add(Dropout(0.4))
    
    model.add(Dense(250,activation = actfn))
    
    model.add(Dropout(0.4))
    
    model.add(Dense(100,activation = actfn))
    
    model.add(Dropout(0.2))
    
    model.add(Dense(50,activation = actfn))
    
    model.add(Dropout(0.1))
    
    model.add(Dense(6,activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy',    
                 optimizer = optimizer_(learning_rate = 0.001),
                 metrics = METRICS) 

    return model

In [ ]:
model = model_cnn()
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [ ]:
fold_no = 1
k_fold = IterativeStratification(n_splits=5, order=6)
for train, test in k_fold.split(X_train, y_train):
    # Compile the model
    
    # Generate a print
    print('-----------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model , class_weight=class_weight
    history = model.fit(X_train[train], y_train[train], class_weight=class_weight, validation_data=(X_train[test], y_train[test]),batch_size=128, epochs=150,callbacks=[custom_early_stopping, reduce_lr])

    scores = model.evaluate(X_train[test], y_train[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    fig, axs = plt.subplots(2)
    fig.suptitle('model loss')
    fig.legend(['train', 'val'], loc='upper left')

    axs[0].plot(history.history['loss'])
    axs[0].plot(history.history['val_loss'])
    axs[1].plot(history.history['auc'])
    axs[1].plot(history.history['val_auc'])

    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
y_pred = model.predict(X_train)
y_pred = np.round(y_pred)
clases = ['No toxic', 'Toxic']
#y_test = y_test.values
vis_arr = metrics.multilabel_confusion_matrix(y_train, y_pred)
labels = ["".join("c" + str(i)) for i in range(0, 6)]
print('ROC AUC: %.3f' % roc_auc_score(y_train, y_pred, average='weighted'))

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12, 7))
    
for axes, cfs_matrix, label in zip(ax.flatten(), vis_arr, labels):
    print_confusion_matrix(np.around(cfs_matrix / cfs_matrix.sum(axis=1)[:, np.newaxis],3), axes, label, ["N", "Y"])
    
fig.tight_layout()
plt.show()